In [1]:
import sys,os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.data.mimic.semi_synthetic_pipeline import MIMICSemiSyntheticDataPipeline
import omegaconf
from hydra.utils import instantiate, get_original_cwd
config_path = '../config/dataset/mimic_synthetic_debug.yaml'

c:\Users\mhr_k\miniconda3\envs\R-learner\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
args = omegaconf.OmegaConf.load(config_path)
#initialize the pipeline
data_pipeline = instantiate(args)

c:\Users\mhr_k\OneDrive\Documents\Thesis\Neural-R-Learner\src\data\mimic\load_data.py:234: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  all_vitals = all_vitals.fillna(method='ffill')
c:\Users\mhr_k\OneDrive\Documents\Thesis\Neural-R-Learner\src\data\mimic\load_data.py:235: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  all_vitals = all_vitals.fillna(method='bfill')
100%|██████████| 10/10 [00:04<00:00,  2.34it/s]
c:\Users\mhr_k\OneDrive\Documents\Thesis\Neural-R-Learner\src\data\mimic\semi_synthetic_pipeline.py:99: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  self.all_vitals = self.all_vitals.unst

InstantiationException: Error in call to target 'src.data.mimic.semi_synthetic_pipeline.MIMICSemiSyntheticDataPipeline':
TypeError('Index does not support mutable operations')

In [3]:
args.keys()


dict_keys(['_target_', 'seed', 'name', 'path', 'min_seq_length', 'max_seq_length', 'max_number', 'data_seed', 'n_periods', 'n_treatments_seq', 'split', 'val_batch_size', 'parallel', 'autoregressive', 'vital_list', 'static_list', 'drop_first', 'synth_outcome', 'synth_treatments_list', 'treatment_outcomes_influence'])